In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings # current version of seaborn generates a bunch of warnings that will be ignore
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

In [4]:
train.drop(['Cabin', 'Name', 'Fare', 'Ticket'], axis=1, inplace=True)
test.drop(['Cabin', 'Name', 'Fare', 'Ticket'], axis=1, inplace=True)

In [5]:
train.dropna(subset=['Embarked'], how='any', inplace=True)
test.dropna(subset=['Embarked'], how='any', inplace=True)

In [6]:
train.Age=train.Age.fillna(value=train.Age.mean())
test.Age=test.Age.fillna(value=train.Age.mean())

In [7]:
Xtrain = train.drop('Survived', axis=1)
ytrain = train['Survived']

Xtest = test
#ytest

In [8]:
genders = {"male": 0, "female": 1}
Xtrain['Sex'] = train['Sex'].map(genders)
Xtest['Sex'] = test['Sex'].map(genders)

In [9]:
ports = {"S": 0, "C": 1, "Q": 2}
Xtrain['Embarked'] = train['Embarked'].map(ports)
Xtest['Embarked'] = test['Embarked'].map(ports)

In [10]:
from sklearn.decomposition import PCA

In [11]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(Xtrain)

In [12]:
from sklearn import model_selection

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [15]:
# Spot check algorithms
models = []
models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

#pca = PCA(n_components=2)
#X_pca = pca.fit_transform(X)

# evaluate each model
results = []
names = []
results_mean = []
for name, model in models:
    model.fit(X_pca, ytrain)
    newdata_transformed = pca.transform(Xtest)
    predictions = model.predict(newdata_transformed)
    cv_results = model_selection.cross_val_score(model, Xtrain, ytrain, scoring = 'accuracy')
    
    results.append(cv_results)
    names.append(name)
    results_mean.append(cv_results.mean())
    msg = "%s: %f " % (name, cv_results.mean())
    print(msg)

LR: 0.799762 
KNN: 0.542095 
CART: 0.690717 
NB: 0.760431 
SVM: 0.615297 


In [16]:
LR = LogisticRegression()
fitting  = LR.fit(Xtrain, ytrain)
predictions = LR.predict(Xtest)

In [17]:
df = pd.DataFrame({'PassengerId': Xtest.PassengerId, 'Survived': predictions}).set_index('PassengerId')

In [18]:
predictions = df.to_csv('submission2.csv')